In [1]:
import os
import numpy as np
import tensorflow as tf
import kagglehub
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
dataset_path = kagglehub.dataset_download("manmandes/malimg")

#Set correct dataset path
dataset_path = os.path.join(dataset_path, "malimg_dataset", "train")  # Ensure we target 'train' folder

print(f" Dataset loaded from: {dataset_path}")

# Step 3: Verify dataset contents
# if not os.path.exists(dataset_path) or not os.listdir(dataset_path):
#     raise ValueError(" No images found! Check the dataset path.")

 Dataset loaded from: /root/.cache/kagglehub/datasets/manmandes/malimg/versions/1/malimg_dataset/train


In [3]:
# Function to load and preprocess the dataset
def load_malimg_dataset(dataset_path, img_size=(64, 64)):
    images, labels = [], []
    class_names = sorted(os.listdir(dataset_path))
    class_dict = {name: i for i, name in enumerate(class_names)}

    for class_name in class_names:
        class_dir = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                try:
                    img = load_img(img_path, target_size=img_size, color_mode="grayscale")
                    img_array = img_to_array(img) / 255.0
                    images.append(img_array)
                    labels.append(class_dict[class_name])
                except Exception as e:
                    print(f" Skipping {img_path}: {e}")

    if len(images) == 0:
        raise ValueError(" No images loaded. Check dataset path and format!")

    X = np.array(images).reshape(-1, img_size[0], img_size[1], 1)
    y = np.array(labels)
    y = to_categorical(y, num_classes=len(class_names))
    return X, y, class_names, class_dict

In [4]:
#  Load dataset
X, y, class_names, class_dict = load_malimg_dataset(dataset_path)
print(f" Dataset size: {X.shape}, Labels: {y.shape}")

 Dataset size: (7459, 64, 64, 1), Labels: (7459, 25)


In [5]:
#  Step 5: Split dataset into Train, Validation, Test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(f" Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")


 Train: (5221, 64, 64, 1), Validation: (1119, 64, 64, 1), Test: (1119, 64, 64, 1)


In [6]:
#  Step 6: Define AlexNet Model with adjusted layers
def create_alexnet(input_shape, num_classes):
    model = keras.Sequential([
        # Change stride to 2, and kernel size to 3x3 to avoid reducing dimensions too quickly
        layers.Conv2D(96, (11, 11), strides=4, activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(256, (5, 5), activation='relu', padding="same"),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(384, (3, 3), activation='relu', padding="same"),
        layers.Conv2D(384, (3, 3), activation='relu', padding="same"),
        layers.Conv2D(256, (3, 3), activation='relu', padding="same"),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [7]:
#  Step 7: Initialize & Train AlexNet
alexnet_model = create_alexnet(input_shape=(64, 64, 1), num_classes=len(class_names))
alexnet_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 16, 16, 96)          │          11,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 7, 7, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 7, 7, 256)           │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 3, 3, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 3, 3, 384)           │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 3, 3, 384)           │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 3, 3, 256)           │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4096)                │       1,052,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 25)                  │         102,425 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,660,377 (82.63 MB)

 Trainable params: 21,660,377 (82.63 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Initialize lists to store accuracy and loss per epoch
epoch_losses = []
epoch_accuracies = []

# #  Train the model
# history = alexnet_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))
# # Collect loss and accuracy for each epoch
# for epoch in range(len(history.history['loss'])):
#     epoch_losses.append(history.history['loss'][epoch])
#     epoch_accuracies.append(history.history['accuracy'][epoch])

# # Calculate the sum of accuracy and loss over all epochs
# total_loss = np.sum(epoch_losses)
# total_accuracy = np.sum(epoch_accuracies)

# # Calculate the average (mean) of loss and accuracy
# mean_loss = np.mean(epoch_losses)
# mean_accuracy = np.mean(epoch_accuracies)

# # Print the results
# print(f"✅ Total Loss over all epochs: {total_loss:.4f}")
# print(f"✅ Total Accuracy over all epochs: {total_accuracy:.4f}")
# print(f"✅ Mean Loss over all epochs: {mean_loss:.4f}")
# print(f"✅ Mean Accuracy over all epochs: {mean_accuracy:.4f}")


# ✅ Train the model and get the training history
history = alexnet_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Calculate the average loss and accuracy over all epochs
average_loss = np.mean(history.history['loss'])
average_accuracy = np.mean(history.history['accuracy'])

# Print the average loss and accuracy
print(f"✅ Average Loss over all epochs: {average_loss:.4f}")
print(f"✅ Average Accuracy over all epochs: {average_accuracy:.4f}")


Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9988 - loss: 0.0040 - val_accuracy: 0.9669 - val_loss: 0.2305
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9980 - loss: 0.0073 - val_accuracy: 0.9705 - val_loss: 0.2798
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9974 - loss: 0.0087 - val_accuracy: 0.9696 - val_loss: 0.2455
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9991 - loss: 0.0045 - val_accuracy: 0.9732 - val_loss: 0.2257
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9997 - loss: 0.0020 - val_accuracy: 0.9723 - val_loss: 0.3040
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9665 - loss: 0.2595 - val_accuracy: 0.9616 - val_loss: 0.3189
Epoch 7/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9871 - loss: 0.0637 - val_accuracy: 0.9696 - val_loss: 0.2694
Epoch 8/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9922 - loss: 0.0306 - val_accu